### Load and preprocess the TRAINING data

In [7]:
import pandas as pd 

df = pd.read_csv("Training.csv")
df = df.drop("Unnamed: 133", axis = 1)

print("Shape of dataset {}".format(df.shape))
df.head()

X_training = df.drop("prognosis",axis = 1)
y = df["prognosis"]

Shape of dataset (4920, 133)


### Data Split 

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

### Model Selection

In [3]:
from sklearn.neighbors import KNeighborsClassifier
